<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study3/blob/main/1D_cnn_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [3]:
# 기본 경로 설정
base_dir = '/content/drive/MyDrive/ship_data'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']

# 데이터 로드 및 전처리 함수 정의
def load_data(base_dir, split):
    X = []
    y = []
    split_dir = os.path.join(base_dir, split)
    for category in categories:
        category_dir = os.path.join(split_dir, category)
        for file in os.listdir(category_dir):
            file_path = os.path.join(category_dir, file)
            data = pd.read_csv(file_path, header=None).values
            data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, data.shape[1])
            data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
            X.append(data)
            y.append(category)
    return np.array(X), np.array(y)

# 데이터 로드
X_train, y_train = load_data(base_dir, 'train')
X_val, y_val = load_data(base_dir, 'validation')
X_test, y_test = load_data(base_dir, 'test')

# 데이터 차원 변경 (CNN 입력 형식에 맞게)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# 원-핫 인코딩
y_train_categorical = to_categorical(y_train_encoded)
y_val_categorical = to_categorical(y_val_encoded)
y_test_categorical = to_categorical(y_test_encoded)

In [4]:
from tensorflow.keras import layers, models

def create_cnn_model():
    model = models.Sequential()

    # Input layer is implicit in Keras; define input shape in first layer
    model.add(layers.InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])))

    # Convolution layer 1
    model.add(layers.Conv1D(filters=16, kernel_size=201, strides=1, activation='relu'))
    # Pooling layer 1
    model.add(layers.AveragePooling1D(pool_size=2, strides=1))

    # Convolution layer 2
    model.add(layers.Conv1D(filters=64, kernel_size=101, strides=1, activation='relu'))
    # Pooling layer 2
    model.add(layers.AveragePooling1D(pool_size=2, strides=1))

    # Convolution layer 3
    model.add(layers.Conv1D(filters=128, kernel_size=51, strides=1, activation='relu'))
    # Pooling layer 3
    model.add(layers.AveragePooling1D(pool_size=2, strides=1))

    # Convolution layer 4
    model.add(layers.Conv1D(filters=256, kernel_size=31, strides=1, activation='relu'))
    # Pooling layer 4
    model.add(layers.AveragePooling1D(pool_size=2, strides=1))

    # Flatten layer
    model.add(layers.Flatten())

    # Fully connected layer
    model.add(layers.Dense(256, activation='relu'))

    # Output layer
    model.add(layers.Dense(len(categories), activation='softmax'))

    return model

model = create_cnn_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 11801, 16)         6448      
                                                                 
 average_pooling1d (Average  (None, 11800, 16)         0         
 Pooling1D)                                                      
                                                                 
 conv1d_1 (Conv1D)           (None, 11700, 64)         103488    
                                                                 
 average_pooling1d_1 (Avera  (None, 11699, 64)         0         
 gePooling1D)                                                    
                                                                 
 conv1d_2 (Conv1D)           (None, 11649, 128)        417920    
                                                                 
 average_pooling1d_2 (Avera  (None, 11648, 128)        0

In [5]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# 모델 학습
history = model.fit(X_train, y_train_categorical,
                    epochs=20,
                    batch_size=32,
                    validation_data=(X_val, y_val_categorical))

Epoch 1/20
219/219 [==============================] - 1407s 6s/step - loss: 9.9766 - accuracy: 0.4750 - val_loss: 1.2262 - val_accuracy: 0.5869
Epoch 2/20
219/219 [==============================] - 1407s 6s/step - loss: 1.2516 - accuracy: 0.4997 - val_loss: 1.1525 - val_accuracy: 0.5869
Epoch 3/20
219/219 [==============================] - 1408s 6s/step - loss: 1.2429 - accuracy: 0.4997 - val_loss: 1.1448 - val_accuracy: 0.5869
Epoch 4/20
219/219 [==============================] - 1406s 6s/step - loss: 1.2432 - accuracy: 0.4997 - val_loss: 1.1475 - val_accuracy: 0.5869
Epoch 5/20
219/219 [==============================] - 1409s 6s/step - loss: 1.2431 - accuracy: 0.4997 - val_loss: 1.1458 - val_accuracy: 0.5869
Epoch 6/20
219/219 [==============================] - 1410s 6s/step - loss: 1.2431 - accuracy: 0.4997 - val_loss: 1.1465 - val_accuracy: 0.5869
Epoch 7/20
219/219 [==============================] - 1414s 6s/step - loss: 1.2430 - accuracy: 0.4997 - val_loss: 1.1490 - val_accuracy:

In [7]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test accuracy: {test_accuracy}')

40/40 [==============================] - 26s 655ms/step - loss: 1.1501 - accuracy: 0.5869
Test accuracy: 0.5868544578552246


In [8]:
# 원본 모델 저장
model.save('model.h5')

# 모델 사이즈 측정
model_size = os.path.getsize('model.h5') / (1024 * 1024)
print(f"Model Size: {model_size:.2f} MB")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model Size: 8730.49 MB


Motor On-Line Fault Diagnosis Method Research Based on 1D-CNN and Multi-Sensor Information